In [1]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [2]:
using Revise
using Turing
using LinearAlgebra
using Distributions
using Random
using StatsPlots
using PDMats
using StatsBase
using Base.Iterators
using DataFrames, TimeSeries, XLSX
using PrettyTables

In [3]:
include("../VARs/utils.jl")
includet("../bootstrap_model.jl")

In [4]:
df = DataFrame(XLSX.readtable("../../data/usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.month, last)

freq = 12
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
start = Date(1955, 01,01)
end_d = Date(2024, 12, 31)


display("Returns")
display(returns_summarystats(to(from(returns, start),end_d), freq))
display("Real_returns")
display(returns_summarystats(to(from(real_returns, start),end_d), freq))


display("Correlations")
display(cor(values(to(from(real_returns, start),end_d)))) 

"Returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.1228,1.3903,0.1011,0.3652,-2.6994,-1.1768,-0.1044,0.2218,4.9165,0.8232,0.1442
Price_10Y,0.0755,1.3969,0.0531,0.1909,-0.9866,0.3097,-0.0893,0.1022,1.9867,0.7037,0.0377


nothing

"Real_returns"

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.1239,1.3604,0.0656,0.3354,-2.5776,-1.0987,-0.1294,0.2287,4.1943,0.5297,0.1107
Price_10Y_CPI,0.078,1.279,0.0177,0.1544,-1.1677,0.2268,-0.1322,0.1437,2.0153,0.2268,0.0058


nothing

"Correlations"

2×2 Matrix{Float64}:
 1.0       0.188888
 0.188888  1.0

In [11]:
assets_names = [:Total_Stock_CPI, :Price_10Y_CPI]
data = to(from(real_returns[assets_names], start),end_d) 

scenarios = block_bootstrap(values(data), 10_000, block_size=1)

periods = [1,5, 10, 25]
ret_in_years = cum_returns_in_periods(scenarios, periods, freq, true)
print_scenarios_summary(ret_in_years, assets_names, string.(periods))
n_assets = length(assets_names)
for a in 1:n_assets
    print_scenarios_percentiles(ret_in_years[a, :, :], [.01, .025, .05, .25, .5, .75, .95, .975, .99], string.(periods), string.(assets_names[a]))
end  


,Total_Stock_CPI,Price_10Y_CPI
1,0.0664,0.0174
5,0.0662,0.0175
10,0.0662,0.0178
25,0.0659,0.0177


,Total_Stock_CPI,Price_10Y_CPI
1,0.1243,0.0779
5,0.0556,0.0348
10,0.0391,0.0246
25,0.0248,0.0157


,Total_Stock_CPI,Price_10Y_CPI
1,-0.2848,0.0373
5,-0.181,0.0145
10,-0.1104,0.0467
25,-0.1226,0.0433


,Total_Stock_CPI,Price_10Y_CPI
1,0.2751,0.1357
5,0.1233,0.0348
10,-0.0098,-0.0112
25,0.0637,-0.0602


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.252,-0.1921,-0.1469,-0.0119,0.0724,0.1516,0.2592,0.292,0.3307
5,-0.0697,-0.0477,-0.0293,0.0303,0.0682,0.1042,0.1536,0.1709,0.1892
10,-0.0279,-0.0123,0.0005,0.0401,0.0669,0.0929,0.1298,0.1404,0.1536
25,0.0057,0.0151,0.0251,0.0495,0.0664,0.0828,0.1059,0.1132,0.1213


,0.01,0.025,0.05,0.25,0.5,0.75,0.95,0.975,0.99
1,-0.1656,-0.1375,-0.1088,-0.0339,0.0167,0.0689,0.1442,0.1734,0.2046
5,-0.0627,-0.0511,-0.0397,-0.0058,0.0174,0.0406,0.0739,0.086,0.0994
10,-0.0394,-0.0295,-0.0218,0.0012,0.0175,0.0344,0.0584,0.0658,0.0761
25,-0.0187,-0.013,-0.0078,0.007,0.0177,0.0281,0.0438,0.0495,0.055


In [10]:
mdd, mddl = max_drawdown_and_length(scenarios[1,:,:])

display(assets_names[1])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

mdd, mddl = max_drawdown_and_length(scenarios[2,:,:])

display(assets_names[2])
display("mdd")
display(quantile(mdd, [.03, .25, .5, .75, .97])')
display("mddl")
display(quantile(mddl, [.03, .25, .5, .75, .97])')

:Total_Stock_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.246152  0.313866  0.36408  0.424876  0.555611

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 45.0  71.0  94.0  128.0  232.0

:Price_10Y_CPI

"mdd"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 0.209271  0.281033  0.337458  0.406685  0.558691

"mddl"

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 90.0  157.0  225.0  331.0  662.0